## ⚙️ Install Required Libraries

Execute the following cell to install or upgrade necessary libraries.

In [ ]:
!pip install huggingface_hub --upgrade
!pip install ipywidgets --upgrade
!pip install git+https://github.com/huggingface/huggingface_hub --upgrade
!pip install hf_transfer --upgrade
!pip install --upgrade jupyterlab-widgets
!pip install --upgrade ipywidgets
!jupyter nbextension enable --py widgetsnbextension

## 🔑 Authenticate with Hugging Face

Enter your Hugging Face token below. You can generate or retrieve your token from [Hugging Face Tokens](https://huggingface.co/settings/tokens).

In [ ]:
import os
import subprocess
import platform

hugging_face_token = input('Enter your Hugging Face token: ')

# Set the environment variable
os.environ['HUGGING_FACE_HUB_TOKEN'] = hugging_face_token
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = "1"
os.environ['HF_HUB_VERBOSITY'] = "info"  # Changed to "info" to reduce verbosity

# Determine the operating system
system = platform.system()

if system == "Linux":
    export_command = f'export HUGGING_FACE_HUB_TOKEN={hugging_face_token}'
    subprocess.run(export_command, shell=True, check=True)
    export_command = f'export HF_HUB_ENABLE_HF_TRANSFER=1'
    subprocess.run(export_command, shell=True, check=True)
    export_command = f'export HF_HUB_VERBOSITY=\"info\"'  # Updated to "info"
    subprocess.run(export_command, shell=True, check=True)

# Command to log in using the token
login_command = ['huggingface-cli', 'login', '--token', hugging_face_token]

# Execute the login command and capture output
try:
    result = subprocess.run(login_command, check=True, capture_output=True, text=True)
    print("Output:", result.stdout)
    print("Error:", result.stderr)
except subprocess.CalledProcessError as e:
    print("Command failed with exit code:", e.returncode)
    print("Output:", e.output)
    print("Error:", e.stderr)

print("🎉 Authentication Successful! 🎉")

## 📥 Download Specific Files

Download one or more specific files from a Hugging Face repository.

In [ ]:
# %% [markdown]
# # Setup and Imports
# This cell imports necessary libraries and defines the base path for ComfyUI models.

# %%
import os
from huggingface_hub import snapshot_download

# --- Base Configuration ---
# !! Verify this is the correct base path to your ComfyUI models directory !!
base_model_path = "/workspace/ComfyUI/models/"
# --- End Configuration ---


# Helper function for downloads
def download_model(repo_id, filenames, destination_subfolder):
    """Downloads specified files from a Hugging Face repo."""
    local_dir = os.path.join(base_model_path, destination_subfolder)
    allow_patterns = [f for f in filenames if f] # Ensure no empty strings

    if not allow_patterns:
        print(f"No valid filenames provided for {repo_id}. Skipping.")
        return

    print("-" * 50)
    print(f"Starting download for: {repo_id}")
    print(f"  Files: {allow_patterns}")
    print(f"  Destination Subfolder: {destination_subfolder}")
    print(f"  Full Path: {local_dir}")

    # Ensure the local directory exists
    try:
        os.makedirs(local_dir, exist_ok=True)
        print(f"Directory '{local_dir}' checked/created.")
    except OSError as e:
        print(f"❌ Error creating directory: {e}")
        print("Please check the base path and permissions.")
        return # Stop if directory fails

    try:
        snapshot_download(
            repo_id=repo_id,
            allow_patterns=allow_patterns,
            local_dir=local_dir,
            local_dir_use_symlinks=False, # Avoid symlinks
            # etag_timeout=60 # Increase timeout if needed
        )
        print(f"✅ DOWNLOAD COMPLETED for {repo_id}")

    except Exception as e:
        print(f"❌ DOWNLOAD FAILED for {repo_id}")
        print(f"Error: {e}")
        print("Please check repository ID, filenames, network connection, and HF token/login status if needed.")
    print("-" * 50)

print("Setup complete. Base model path set to:", base_model_path)
print("Helper function 'download_model' defined.")

In [ ]:
# %% [markdown]
# # Download Flux 1.0 [dev] (UNet Model)
# Downloads the main UNet model for Flux.

# %%
# --- UNet Model ---
repo_id_unet = "black-forest-labs/FLUX.1-dev"
# Check repo for exact filename if needed, e.g., flux1-dev.safetensors or similar
filename_unet = "flux1-dev.safetensors"
dest_folder_unet = "unet"
# ---

download_model(repo_id_unet, [filename_unet], dest_folder_unet)

In [ ]:
# %% [markdown]
# # Download Flux 1.0 Redux [dev] (Style Model)
# Downloads the optional style model.

# %%
# --- Style Model ---
repo_id_style = "black-forest-labs/FLUX.1-Redux-dev"
# !! Check repo files, filename might differ !! Assume 'flux1-redux-dev.safetensors'
filename_style = "flux1-redux-dev.safetensors"
dest_folder_style = "style_models"
# ---

download_model(repo_id_style, [filename_style], dest_folder_style)

In [ ]:
# %% [markdown]
# # Download Text Encoders (T5-XXL & CLIP-L)
# Downloads the required text encoder models.

# %%
# --- Text Encoders ---
repo_id_text = "comfyanonymous/flux_text_encoders"
filenames_text = [
    "t5xxl_fp16.safetensors", # T5-XXL
    "clip_l.safetensors"     # OpenCLIP-Large (CLIP-L)
]
dest_folder_text = "text_encoders"
# ---

download_model(repo_id_text, filenames_text, dest_folder_text)

In [ ]:
# %% [markdown]
# # Download VAE
# Downloads the Variational Autoencoder model.

# %%
# --- VAE Model ---
# Note: This VAE comes from the main Flux dev repo
repo_id_vae = "black-forest-labs/FLUX.1-dev"
filename_vae = "ae.safetensors"
dest_folder_vae = "vae"
# ---

download_model(repo_id_vae, [filename_vae], dest_folder_vae)

In [ ]:
# %% [markdown]
# # Download LoRA - Aidma FLUX Pro 1.1 (Extreme Detailer)
# Downloads the primary detailer LoRA.

# %%
# --- LoRA: Detailer ---
repo_id_lora_detailer = "DavidBaloches/Extreme_Detailer"
filename_lora_detailer = "aidmaFLUXPro1.1-FLUX-v0.3.safetensors"
dest_folder_lora = "loras"
# ---

download_model(repo_id_lora_detailer, [filename_lora_detailer], dest_folder_lora)

In [ ]:
# %% [markdown]
# # Download LoRA - Hyper Realism by Aidma v0.3 (Optional)
# Downloads the optional Hyper Realism LoRA.

# %%
# --- LoRA: Hyper Realism ---
repo_id_lora_hyper = "DavidBaloches/Hyper_Realism_Lora_by_aidma"
# !! Check repo files for exact filename !! Assume 'Hyper_Realism_Lora_by_aidma.safetensors'
filename_lora_hyper = "Hyper_Realism_Lora_by_aidma.safetensors"
dest_folder_lora = "loras"
# ---

download_model(repo_id_lora_hyper, [filename_lora_hyper], dest_folder_lora)

In [ ]:
# %% [markdown]
# # Download Upscaler (4xFaceUpDAT)
# Downloads the face-focused upscaling model.

# %%
# --- Upscaler Model ---
repo_id_upscaler = "RippleLinks/Flux_Models"
filename_upscaler = "4xFaceUpDAT.pth"
dest_folder_upscaler = "upscale_models"
# ---

download_model(repo_id_upscaler, [filename_upscaler], dest_folder_upscaler)